In [ ]:
!pip install transformers torch nltk protobuf pandas

In [1]:
from transformers import pipeline
from nltk import sent_tokenize
import pandas as pd

E:\anaconda3\envs\nlp2025\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
text1 = """Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in
our lives. Hope you too, to enjoy it as my deepest wishes.
Thank your message to show our words to the doctor, as his next contract checking, to all of us.
I got this message to see the approved message. In fact, I have received the message from the
professor, to show me, this, a couple of days ago. I am very appreciated the full support of the
professor, for our Springer proceedings publication"""

text2 = """During our final discuss, I told him about the new submission — the one we were waiting since
last autumn, but the updates was confusing as it not included the full feedback from reviewer or
maybe editor?
Anyway, I believe the team, although bit delay and less communication at recent days, they really
tried best for paper and cooperation. We should be grateful, I mean all of us, for the acceptance
and efforts until the Springer link came finally last week, I think.
Also, kindly remind me please, if the doctor still plan for the acknowledgments section edit before
he sending again. Because I didn’t see that part final yet, or maybe I missed, I apologize if so.
Overall, let us make sure all are safe and celebrate the outcome with strong coffee and future
targets"""

texts = [text1, text2]


In [3]:
first = pipeline("text2text-generation", model="vennify/t5-base-grammar-correction")

second = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

third = pipeline("text2text-generation", model="prithivida/grammar_error_correcter_v1")

Device set to use cpu
Device set to use cpu
Device set to use cpu


In [4]:
results = []

for i, txt in enumerate(texts, start=1):
    print(f"\n--- Text {i} Rewrites ---\n")

    print("Original text",texts[i-1])
    
    sentences = sent_tokenize(txt)
    first_corrected = []
    for s in sentences:
        out = first(s)
        first_corrected.append(out[0]['generated_text'])
    first_corrected_text = " ".join(first_corrected)
    print("First Model Grammar Correction (sentence by sentence):\n", first_corrected_text, "\n")
    
    second_output = second(txt)
    print("Second Model Summarization (full text):\n", second_output[0]['summary_text'], "\n")
    
    third_corrected = []
    for s in sentences:
        out = third(s)
        third_corrected.append(out[0]['generated_text'])
    third_corrected_text = " ".join(third_corrected)
    print("Third Model Grammar Correction (sentence by sentence):\n", third_corrected_text, "\n")

    results.append({
        "Text_ID": i,
        "Original_Text": txt,
        "First_Model_Grammar": first_corrected_text,
        "Second_Model_Summary": second_output[0]['summary_text'],
        "Third_Model_Grammar": third_corrected_text
    })
    
df = pd.DataFrame(results)

df.to_csv("pipeline_outputs.csv", index=False)

print("✅ Results saved to pipeline_outputs.csv")


--- Text 1 Rewrites ---

Original text Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in
our lives. Hope you too, to enjoy it as my deepest wishes.
Thank your message to show our words to the doctor, as his next contract checking, to all of us.
I got this message to see the approved message. In fact, I have received the message from the
professor, to show me, this, a couple of days ago. I am very appreciated the full support of the
professor, for our Springer proceedings publication


Your max_length is set to 142, but your input_length is only 117. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=58)


First Model Grammar Correction (sentence by sentence):
 Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safety and great in our lives. Hope you too, to enjoy it as my deepest wishes. Thank you for your message to show our words to the doctor, as his next contract checking, to all of us. I got this message to see the approved message. In fact, I have received the message from the professor, to show me, this, a couple of days ago. I am very appreciated the full support of the professor, for our Springer proceedings publication. 

Second Model Summarization (full text):
  It is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives, hope you too, to enjoy it as my deepest wishes . I am very appreciated the full support of the . full support . for our Springer proceedings publication . I got this message to see the approved message . 

Third Model Grammar Correction (sentence by sentence):
 Today is our dra